# Setting Up and Testing kubectl to control a kubernetes cluster

# Part 1 - Started during Workshop Slides

For this module, the Nautilus Jupyter Lap environment is our working environment.  
To control the Kubernetes cluster, please use the **Stack Datascience**.

## 1.0 Get the Nautilus Kubernetes Configuration File

https://nautilus.optiputer.net/

![Nautilus_get_config.png MISSING](../images/Nautilus_get_config.png)

After you get the file downloaded, please follow directions they provide!
![Nautilus_get_config_post_download.png MISSING](../images/Nautilus_get_config_post_download.png)

### The simple way to put the `config` into your Nautilus Jupyter Lab

1. Upload into the top-level (home) folder.
2. Create the kubernetes config folder, and then move the config into that folder.
```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ mkdir ~/.kube
jovyan@jupyter-scottgs-40missouri-2eedu:~$ mv config ~/.kube/
```


---

## 1.1 Install kubectl into your container on your Nautilus Jupyter Lab 

### Linux

https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/

Specifically **_Install using native package management_**:   
https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#install-using-native-package-management

#### Please confirm that you follow the steps and see the `Setting up kubectl (1.23.4-00)` or at least some version that starts with 1.23

```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ sudo apt-get install -y kubectl
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  kubectl
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 8,927 kB of archives.
After this operation, 46.6 MB of additional disk space will be used.
Get:1 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 kubectl amd64 1.23.4-00 [8,927 kB]
Fetched 8,927 kB in 0s (21.4 MB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package kubectl.
(Reading database ... 63512 files and directories currently installed.)
Preparing to unpack .../kubectl_1.23.4-00_amd64.deb ...
Unpacking kubectl (1.23.4-00) ...
Setting up kubectl (1.23.4-00) ...
```


### Test the Confg setup
`kubectl cluster-info -n gpn-mizzou-tutorial`

### Alternative System Install Guides
#### Windows
https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/

#### Mac

https://kubernetes.io/docs/tasks/tools/install-kubectl-macos/


### Before you continue, review some of the Nautilus documentation
 * https://ucsd-prp.gitlab.io/userdocs/start/quickstart/
 * https://ucsd-prp.gitlab.io/userdocs/tutorial/basic/
 
Of particular note, we need to set the default namespace to `gpn-mizzou-tutorial`

```


# Part 2: Execute a simple pod

As we start working on these commands, we will need to change our terminal position to be the workshop folder:
```BASH
cd Nautilus_Kubernetes_DeepLearning
```

The result should similar to:
```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ cd Nautilus_Kubernetes_DeepLearning/
jovyan@jupyter-scottgs-40missouri-2eedu:~/Nautilus_Kubernetes_DeepLearning$ 
```

## 2.1 Write yaml file for the pod

 Open the file `resources/pod_python.yml`.

 Edit file and replace SSO in line 4 with your Name
 
 ![Nautilus_kube_lab_pod_yaml.png MISSING](../images/Nautilus_kube_lab_pod_yaml.png)

## 2.2 Run the pod

```BASH
kubectl -n gpn-mizzou-tutorial create -f resources/pod_python.yml
```

### Progress Check:  Did you get a response similar to:
```
pod/pod-python-scottgs created
```

## 2.3 Check pod 

 Run the command below, it will show you the status of your pod
```BASH
kubectl -n gpn-mizzou-tutorial get pods
```

Keep checking the pod status until you see the pod is running!
If the pod gets into an error status or stuck somehow, remote it (step 2.9), check your YAML, and go to Step 2.2.
```
jovyan@jupyter-scottgs-40missouri-2eedu:~/Nautilus_Kubernetes_DeepLearning$ kubectl -n gpn-mizzou-tutorial get pods
NAME                     READY   STATUS    RESTARTS   AGE
job-copy-data--1-mqz2q   1/1     Running   0          5h36m
pod-name-sso             1/1     Running   0          4h42m
pod-python-scottgs       1/1     Running   0          58s
```

 ## 2.4 Copy code and image files to _your_ pod

For each file referenced below, you will need copy it from the `resources` in your Nautilus Jupyter Lab instance.
Example directory listing on Nautilus Jupyter of uploaded files.

```
process_image.py
image1.npy
image2.npy
pod_python.yml
```

 * Make sure you update the `pod_name` to be your actual pod name.

```BASH
kubectl -n gpn-mizzou-tutorial cp resources/process_image.py pod_name:home/process_image.py
kubectl -n gpn-mizzou-tutorial cp resources/image1.npy pod_name:home/image1.npy
kubectl -n gpn-mizzou-tutorial cp resources/image2.npy pod_name:home/image2.npy
```

## 2.5  Access the pod, look for our copied files.

Now we will get interactive access on pod using the command below, where the `/bin/bash` is our terminal shell.

```BASH
kubectl -n gpn-mizzou-tutorial exec -it pod_name -- /bin/bash

cd home

ls -lt
```

 #### Example / Expected view
 
```BASH
jovyan@jupyter-scottgs-40missouri-2eedu:~$ kubectl -n gpn-mizzou-tutorial exec -it pod-python-scottgs -- /bin/bash
root@pod-python-scottgs:/#
root@pod-python-scottgs:/# cd home
root@pod-python-scottgs:/home# ls -lt
total 5372
-rw-r--r-- 1 1000 users  422628 Mar  2 05:13 image2.npy
-rw-r--r-- 1 1000 users 5070128 Mar  2 05:13 image1.npy
-rw-r--r-- 1 1000 users     300 Mar  2 05:13 process_image.py
```

## 2.6 Install libraries in your pod

```BASH
pip install numpy  matplotlib pillow
```

## 2.7 Run Python code 

```BASH
cd /home
python3 process_image.py 
```

#### Check Point: After running the program (python code) you should have a new file:
```BASH
root@pod-python-scottgs:/home# ls -lt | head
total 5488
-rw-r--r-- 1 root root   118118 Mar  2 05:21 images.png
```


## 2.8 Exit the Pod and Copy resulting image back to your local repo

To exit the pod, you can type `exit` or use `[Control]+D` for _disconnect_.  
Then, proceed to pull the result image down.

```BASH
kubectl -n gpn-mizzou-tutorial cp pod_name:home/images.png output_image.png
```

## 2.9 Delete pod


```BASH
kubectl -n gpn-mizzou-tutorial delete pod pod_name
```

#### <span style='background:blue'>Did you see a message similar to this?</span>
```
pod "pod-python-scottgs" deleted
```

---

# Notebook Complete

You have learned how to
 * spin up pods, 
 * copy files to and from pods, 
 * access the running pod
 * clean up pods